In [16]:
# %matplotlib widget

from __future__ import annotations

import re
from collections import defaultdict
from dataclasses import dataclass, field
from itertools import permutations, product
from math import inf
from random import choice

import matplotlib.colors as mcolors
import matplotlib.pyplot as plt
import networkx as nx
import numpy as np
import numpy.typing as npt
from mpl_toolkits.mplot3d import axes3d
from numpy import int_, object_
from numpy.typing import NDArray
from test_utilities import test
from util import *

COLORS = list(mcolors.CSS4_COLORS.keys())

<link href="style.css" rel="stylesheet"></link>
<article class="day-desc"><h2>--- Day 14: Restroom Redoubt ---</h2><p>One of The Historians needs to use the bathroom; fortunately, you know there's a bathroom near an unvisited location on their list, and so you're all quickly teleported directly to the lobby of Easter Bunny Headquarters.</p>
<p>Unfortunately, EBHQ seems to have "improved" bathroom security <em>again</em> after your last <a href="/2016/day/2">visit</a>. The area outside the bathroom is swarming with robots!</p>
<p>To get The Historian safely to the bathroom, you'll need a way to predict where the robots will be in the future. Fortunately, they all seem to be moving on the tile floor in predictable <em>straight lines</em>.</p>
<p>You make a list (your puzzle input) of all of the robots' current <em>positions</em> (<code>p</code>) and <em>velocities</em> (<code>v</code>), one robot per line. For example:</p>
<pre><code>p=0,4 v=3,-3
p=6,3 v=-1,-3
p=10,3 v=-1,2
p=2,0 v=2,-1
p=0,0 v=1,3
p=3,0 v=-2,-2
p=7,6 v=-1,-3
p=3,0 v=-1,-2
p=9,3 v=2,3
p=7,3 v=-1,2
p=2,4 v=2,-3
p=9,5 v=-3,-3
</code></pre>
<p>Each robot's position is given as <code>p=x,y</code> where <code>x</code> represents the number of tiles the robot is from the left wall and <code>y</code> represents the number of tiles from the top wall (when viewed from above). So, a position of <code>p=0,0</code> means the robot is all the way in the top-left corner.</p>
<p>Each robot's velocity is given as <code>v=x,y</code> where <code>x</code> and <code>y</code> are given in <em>tiles per second</em>. Positive <code>x</code> means the robot is moving to the <em>right</em>, and positive <code>y</code> means the robot is moving <em>down</em>. So, a velocity of <code>v=1,-2</code> means that each second, the robot moves <code>1</code> tile to the right and <code>2</code> tiles up.</p>
<p>The robots outside the actual bathroom are in a space which is <code>101</code> tiles wide and <code>103</code> tiles tall (when viewed from above). However, in this example, the robots are in a space which is only <code>11</code> tiles wide and <code>7</code> tiles tall.</p>
<p>The robots are good at navigating over/under each other (due to a combination of springs, extendable legs, and quadcopters), so they can share the same tile and don't interact with each other. Visually, the number of robots on each tile in this example looks like this:</p>
<pre><code>1.12.......
...........
...........
......11.11
1.1........
.........1.
.......1...
</code></pre>
<p>These robots have a unique feature for maximum bathroom security: they can <em>teleport</em>. When a robot would run into an edge of the space they're in, they instead <em>teleport to the other side</em>, effectively wrapping around the edges. Here is what robot <code>p=2,4 v=2,-3</code> does for the first few seconds:</p>
<pre><code>Initial state:
...........
...........
...........
...........
..1........
...........
...........

After 1 second:
...........
....1......
...........
...........
...........
...........
...........

After 2 seconds:
...........
...........
...........
...........
...........
......1....
...........

After 3 seconds:
...........
...........
........1..
...........
...........
...........
...........

After 4 seconds:
...........
...........
...........
...........
...........
...........
..........1

After 5 seconds:
...........
...........
...........
.1.........
...........
...........
...........
</code></pre>

<p>The Historian can't wait much longer, so you don't have to simulate the robots for very long. Where will the robots be after <code>100</code> seconds?</p>
<p>In the above example, the number of robots on each tile after 100 seconds has elapsed looks like this:</p>
<pre><code>......2..1.
...........
1..........
.11........
.....1.....
...12......
.1....1....
</code></pre>
<p>To determine the safest area, count the <em>number of robots in each quadrant</em> after 100 seconds. Robots that are exactly in the middle (horizontally or vertically) don't count as being in any quadrant, so the only relevant robots are:</p>
<pre><code>..... 2..1.
..... .....
1.... .....
           
..... .....
...12 .....
.1... 1....
</code></pre>
<p>In this example, the quadrants contain <code>1</code>, <code>3</code>, <code>4</code>, and <code>1</code> robot. Multiplying these together gives a total <em>safety factor</em> of <code><em>12</em></code>.</p>
<p>Predict the motion of the robots in your list within a space which is <code>101</code> tiles wide and <code>103</code> tiles tall. <em>What will the safety factor be after exactly 100 seconds have elapsed?</em></p>
</article>


In [17]:
from math import prod


tests = [
    {
        "name": "Single Robot",
        "s": """
           p=2,4 v=2,-3
        """,
        "seconds": 5,
        "expected": 0,
    },
    {
        "name": "Example",
        "s": """
            p=0,4 v=3,-3
            p=6,3 v=-1,-3
            p=10,3 v=-1,2
            p=2,0 v=2,-1
            p=0,0 v=1,3
            p=3,0 v=-2,-2
            p=7,6 v=-1,-3
            p=3,0 v=-1,-2
            p=9,3 v=2,3
            p=7,3 v=-1,2
            p=2,4 v=2,-3
            p=9,5 v=-3,-3
        """,
        "seconds": 100,
        "expected": 12,
    },
]


class BathRoom(Str):
    def __init__(self, s: str, wide: int, tall: int) -> None:
        self.room = [[[] for _ in range(wide)] for _ in range(tall)]
        self.empty = [[[] for _ in range(wide)] for _ in range(tall)]
        self._place_robots(s)

    def run_for(self, seconds: int, debug=False) -> None:
        if debug:
            print(f"Initial state:\n{self}\n")

        for i in range(seconds):
            self.tick()
            if debug:
                print(f"After {i+1} second::\n{self}\n")

    def tick(self) -> None:
        rows, cols = len(self.room), len(self.room[0])

        for r, c in product(range(rows), range(cols)):
            for dr, dc in self.room[r][c]:
                self.empty[(r + dr) % rows][(c + dc) % cols].append((dr, dc))

            self.room[r][c].clear()

        self.room, self.empty = self.empty, self.room

    def safety_factor(self) -> int:
        rows, cols = len(self.room), len(self.room[0])
        mid_row, mid_col = rows // 2, cols // 2
        return prod(
            (
                sum(
                    len(self.room[r][c])
                    for r, c in product(range(mid_row), range(mid_col))
                ),
                sum(
                    len(self.room[r][c])
                    for r, c in product(range(mid_row), range(mid_col + 1, cols))
                ),
                sum(
                    len(self.room[r][c])
                    for r, c in product(range(mid_row + 1, rows), range(mid_col))
                ),
                sum(
                    len(self.room[r][c])
                    for r, c in product(
                        range(mid_row + 1, rows), range(mid_col + 1, cols)
                    )
                ),
            )
        )

    def _place_robots(self, s: str) -> None:
        for line in s.strip().splitlines():
            c, r, dc, dr = map(int, re.findall(r"-?\d+", line))
            self.room[r][c].append((dr, dc))

    def __str__(self) -> str:
        return "\n".join(
            "".join(f"{len(c) if c else '.'}" for c in r) for r in self.room
        )

    @classmethod
    def partI(cls, s: str, seconds: int, wide: int, tall: int, debug=False) -> int:
        br = cls(s, wide=wide, tall=tall)
        br.run_for(seconds=seconds, debug=debug)
        sf = br.safety_factor()
        return sf


@test(tests=tests[:])
def partI_test(s: str, seconds: int) -> int:
    sf = BathRoom.partI(s, seconds, wide=11, tall=7)
    return sf


Test Single Robot passed, for partI_test.
Test Example passed, for partI_test.
Success


In [18]:
with open("../input/day14.txt") as f:
    puzzle = f.read()

print(f"Part I: {BathRoom.partI(puzzle,seconds=100,wide=101,tall=103)}")

Part I: 229868730


<link href="style.css" rel="stylesheet"></link>
<main>

<p>Your puzzle answer was <code>29517</code>.</p><p class="day-success">The first half of this puzzle is complete! It provides one gold star: *</p>


<link href="style.css" rel="stylesheet"></link>
<article class="day-desc"><h2 id="part2">--- Part Two ---</h2><p>During the bathroom break, someone notices that these robots seem awfully similar to ones built and used at the North Pole. If they're the same type of robots, they should have a hard-coded <span title="This puzzle was originally going to be about the motion of space rocks in a fictitious arcade game called Meteoroids, but we just had an arcade puzzle.">Easter egg</span>: very rarely, most of the robots should arrange themselves into <em>a picture of a Christmas tree</em>.</p>
<p><em>What is the fewest number of seconds that must elapse for the robots to display the Easter egg?</em></p>
</article>


In [19]:
class BathRoomII(BathRoom):
    def run_till(self, debug=False) -> int:
        # Slow as hell, but it works
        i = 1
        while True:
            self.tick()
            if "***********" in str(self):
                if debug:
                    print(f"After {i} second::\n{self}\n")
                return i

            i += 1

    def __str__(self) -> str:
        return "\n".join("".join(f"{'*'if c else '.'}" for c in r) for r in self.room)

    @classmethod
    def partII(cls, s: str, wide: int, tall: int, debug=False) -> int:
        br = cls(s, wide=wide, tall=tall)
        return br.run_till(debug=debug)


print(f"Part II: { BathRoomII.partII(puzzle, wide=101, tall=103, debug=True)}")

After 7861 second::
................................................*.......................................*............
*.............................................**....*................................................
.....................................................................................................
..............................................*......................*...............................
...............................*.....................................................................
.........*...*............................................................................*..........
..............................................*......................................................
....*.....................*........................................................................*.
............................*........................................................................
............*.................................................

<link href="style.css" rel="stylesheet"></link>


<link href="style.css" rel="stylesheet"></link>
<main>

<p>Your puzzle answer was <code>7861 </code>.</p><p class="day-success">Both parts of this puzzle are complete! They provide two gold stars: **</p>

</main>
